<a href="https://colab.research.google.com/github/mishra-yogendra/DeepLearning/blob/main/CNN/Cat_Vs_Dog_Image_Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
import os
# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dogs-vs-cats


In [2]:
BASE_DIR='/kaggle/input/dogs-vs-cats'
train_dir_path=os.path.join(BASE_DIR,'train')
validation_dir_path=os.path.join(BASE_DIR,'test')

In [3]:
print(train_dir_path)
print(validation_dir_path)

/kaggle/input/dogs-vs-cats/train
/kaggle/input/dogs-vs-cats/test


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

In [5]:
# generators
train_ds =  keras.utils.image_dataset_from_directory(
    directory = train_dir_path,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
)

Found 20000 files belonging to 2 classes.


In [6]:
validation_ds =  keras.utils.image_dataset_from_directory(
    directory = validation_dir_path,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
)

Found 5000 files belonging to 2 classes.


In [7]:
# Normalize
def process(image, label):
    image = tf.cast(image/255. ,tf.float32)
    return image, label
train_ds = train_ds.map(process)
val_ds = validation_ds.map(process)

In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Image size and base model
IMAGE_SIZE = (224, 224)  # Required for MobileNetV2
BATCH_SIZE = 32

# Load pre-trained MobileNetV2
base_model = MobileNetV2(input_shape=IMAGE_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)  # Target size for model input
train_dir_path = train_dir_path
validation_dir_path = validation_dir_path

# Create generators - directly resize to target size
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators with target size = model input size
train_data = train_datagen.flow_from_directory(
    train_dir_path,
    target_size=IMAGE_SIZE,  # Directly resize to model input size
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    validation_dir_path,
    target_size=IMAGE_SIZE,  # Directly resize to model input size
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Use the generators directly - no need for Dataset conversion
# model.fit(train_data, epochs=20, validation_data=val_data, ...)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [10]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6)

History = model.fit(train_data ,epochs=15,validation_data=val_data,    callbacks=[early_stop, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 401s 622ms/step - accuracy: 0.8919 - loss: 0.2507 - val_accuracy: 0.9762 - val_loss: 0.0646 - learning_rate: 1.0000e-04
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 261s 418ms/step - accuracy: 0.9615 - loss: 0.0976 - val_accuracy: 0.9770 - val_loss: 0.0630 - learning_rate: 1.0000e-04
Epoch 3/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 416ms/step - accuracy: 0.9638 - loss: 0.0902 - val_accuracy: 0.9766 - val_loss: 0.0602 - learning_rate: 1.0000e-04
Epoch 4/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 269s 431ms/step - accuracy: 0.9657 - loss: 0.0855 - val_accuracy: 0.9772 - val_loss: 0.0588 - learning_rate: 1.0000e-04
Epoch 5/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 416ms/step - accuracy: 0.9656 - loss: 0.0818 - val_accuracy: 0.9802 - val_loss: 0.0526 - learning_rate: 1.0000e-04
Epoch 6/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 260s 413ms/step - accuracy: 0.9699 - loss: 0.0772 - val_accuracy: 0.9798 - val_loss: 0.0520 - learning_rate: 1.0000e-04
Epoch 7/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 